In [1]:
# test: Query a form from fulcrum (Preventive Maintenance) and publish to postgresql
# import packages
import pandas as pd
import numpy as np
import knackpy as kp
import fulcrum as fc
import requests
import pdb
from datetime import datetime, timedelta
# import credentials
from config.secrets import *

In [2]:
form_id = "44359e32-1a7f-41bd-b53e-3ebc039bd21a"
key = FULCRUM_CRED.get("api_key")

In [3]:
fulcrum = fc.Fulcrum(key = key)

In [4]:
forms = fulcrum.forms.search(url_params={'id': form_id})

In [5]:
records_dirty = fulcrum.records.search(url_params = {"form_id":form_id})

In [6]:
type(records_dirty)

dict

In [7]:
type(records_dirty["records"])

list

In [13]:
print(records_dirty["records"])

[{'status': None, 'version': 1, 'id': 'a91eae80-1723-41bf-9828-e51ef8525648', 'created_at': '2018-08-07T14:15:16Z', 'updated_at': '2018-08-07T14:15:16Z', 'client_created_at': '2018-08-07T12:03:09Z', 'client_updated_at': '2018-08-07T13:12:13Z', 'created_by': 'Signal Tech', 'created_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'updated_by': 'Signal Tech', 'updated_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'created_location': {'latitude': 30.4629025, 'longitude': -97.8434215, 'altitude': None, 'horizontal_accuracy': 40.989}, 'updated_location': {'latitude': 30.4628551, 'longitude': -97.8433521, 'altitude': 272.0, 'horizontal_accuracy': 12.275}, 'created_duration': 1598, 'updated_duration': 3, 'edited_duration': 1604, 'form_id': '44359e32-1a7f-41bd-b53e-3ebc039bd21a', 'project_id': None, 'assigned_to': None, 'assigned_to_id': None, 'form_values': {'4de9': 'OK', '6080': '1017 |  ANDERSON MILL RD / CASHELL WOOD DR', 'ffc1': 'OK', '33cc': 'EXCELLENT', '3f80': 'no', '146e': 'OK', '784

In [8]:
print(records_dirty["records"][0]["form_values"])

{'4de9': {'other_values': [], 'choice_values': ['OK']}, '6080': {'other_values': [], 'choice_values': ['1017 |  ANDERSON MILL RD / CASHELL WOOD DR']}, 'ffc1': {'other_values': [], 'choice_values': ['OK']}, '33cc': {'other_values': [], 'choice_values': ['EXCELLENT']}, '3f80': 'no', '146e': {'other_values': [], 'choice_values': ['OK']}, '7840': {'other_values': [], 'choice_values': ['No Preemption']}, '93dd': {'other_values': [], 'choice_values': ['OK']}, 'ecf2': {'other_values': [], 'choice_values': ['OK']}, '6e0a': {'other_values': [], 'choice_values': ['APS']}, 'e1b4': 'yes', 'ea0f': 'no', 'fce3': {'other_values': [], 'choice_values': ['thomas.henry@austintexas.gov', 'toby.turner@austintexas.gov']}, 'f36f': {'other_values': [], 'choice_values': ['No Obstruction']}, 'f283': [{'updated_at': '1533643768', 'created_location': {'latitude': 30.4627221, 'longitude': -97.8434027, 'altitude': 271.0, 'horizontal_accuracy': 10.0}, 'created_duration': 179, 'id': 'cf2e5533-ed13-4eaf-bd94-570c961ad

In [16]:
records_dirty["records"][0]

{'status': None,
 'version': 1,
 'id': 'a91eae80-1723-41bf-9828-e51ef8525648',
 'created_at': '2018-08-07T14:15:16Z',
 'updated_at': '2018-08-07T14:15:16Z',
 'client_created_at': '2018-08-07T12:03:09Z',
 'client_updated_at': '2018-08-07T13:12:13Z',
 'created_by': 'Signal Tech',
 'created_by_id': '1620c175-017a-4f60-b808-30860e663c50',
 'updated_by': 'Signal Tech',
 'updated_by_id': '1620c175-017a-4f60-b808-30860e663c50',
 'created_location': {'latitude': 30.4629025,
  'longitude': -97.8434215,
  'altitude': None,
  'horizontal_accuracy': 40.989},
 'updated_location': {'latitude': 30.4628551,
  'longitude': -97.8433521,
  'altitude': 272.0,
  'horizontal_accuracy': 12.275},
 'created_duration': 1598,
 'updated_duration': 3,
 'edited_duration': 1604,
 'form_id': '44359e32-1a7f-41bd-b53e-3ebc039bd21a',
 'project_id': None,
 'assigned_to': None,
 'assigned_to_id': None,
 'form_values': {'4de9': 'OK',
  '6080': '1017 |  ANDERSON MILL RD / CASHELL WOOD DR',
  'ffc1': 'OK',
  '33cc': 'EXCELLE

In [18]:
records = pd.DataFrame()

for record in records_dirty["records"]:
    form_values = record["form_values"]
    for key, value in form_values.items():
        print(value)
        if type(value) == dict and "choice_values" in value:
            value = value["choice_values"]
            
        if type(value) == list and len(value) == 1:
            form_values[key] = value[0]
            
    form_values["_server_updated_at"] = record["created_at"]
    form_values["_record_id"] = record["id"]
    new_row = pd.DataFrame([form_values], columns=form_values.keys())
#     print(new_row)
    records = pd.concat([new_row, records], axis =0, sort=False).reset_index(drop=True)
    


OK
1017 |  ANDERSON MILL RD / CASHELL WOOD DR
OK
EXCELLENT
no
OK
No Preemption
OK
OK
APS
yes
no
{'other_values': [], 'choice_values': ['thomas.henry@austintexas.gov', 'toby.turner@austintexas.gov']}
No Obstruction
[{'updated_at': '1533643768', 'created_location': {'latitude': 30.4627221, 'longitude': -97.8434027, 'altitude': 271.0, 'horizontal_accuracy': 10.0}, 'created_duration': 179, 'id': 'cf2e5533-ed13-4eaf-bd94-570c961ad130', 'updated_location': {'latitude': 30.4627457, 'longitude': -97.8433499, 'altitude': 274.0, 'horizontal_accuracy': 10.0}, 'created_at': '1533643569', 'edited_duration': 188, 'form_values': {'48b8': {'other_values': [], 'choice_values': ['Loop']}, 'fd6a': {'other_values': [], 'choice_values': ['Other']}, 'b740': {'other_values': [], 'choice_values': ['OK']}, 'e332': {'other_values': [], 'choice_values': ['Eastbound']}}, 'updated_duration': 9, 'geometry': None, 'version': 1, 'created_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'updated_by_id': '1620c175-017a-

OK
13:29
13:17
Replaced
Cleaned
2018-08-29T18:21:54Z
OK
172 |  AIRPORT BLVD / KOENIG LN SVRD
OK
GOOD
yes
no
OK
No Preemption
OK
OK
1. Ups is dead needs batteries 
yes
APS
yes
toby.turner@austintexas.gov
No Obstruction
[{'updated_at': '1535479849', 'created_location': {'latitude': 30.3201789, 'longitude': -97.7128371, 'altitude': 167.0, 'horizontal_accuracy': 11.413}, 'created_duration': 21, 'id': '7587c623-edf6-4db9-aee9-c2db1b4158a3', 'updated_location': {'latitude': 30.3201777, 'longitude': -97.712838, 'altitude': 167.0, 'horizontal_accuracy': 10.0}, 'created_at': '1535479828', 'edited_duration': 21, 'form_values': {'48b8': {'other_values': [], 'choice_values': ['Radar']}, 'fd6a': {'other_values': [], 'choice_values': ['Thru / Left Turn']}, 'b740': {'other_values': [], 'choice_values': ['OK']}, 'e332': {'other_values': [], 'choice_values': ['Northbound']}}, 'updated_duration': 21, 'version': 1, 'created_by_id': '1620c175-017a-4f60-b808-30860e663c50', 'updated_by_id': '1620c175-017a-4

In [12]:
print(records[])

   4de9                                               21c6  \
0    OK  {'other_values': ['could not test '], 'choice_...   
1    OK                                                NaN   
2    OK                                                NaN   
3    OK                                                NaN   
4    OK                                                NaN   
5    OK                                                NaN   
6    OK                                                NaN   
7    OK                                                NaN   
8    OK                                                NaN   
9    OK                                                NaN   
10   OK                                                NaN   
11   OK                                                NaN   
12   OK                                                NaN   
13   OK                                                NaN   
14   OK                                                NaN   
15   OK 

In [121]:
records.head(5)

,4de9,21c6,6080,ffc1,33cc,3f80,a460,146e,7840,93dd,...,4a48,c01c,3a14,eb53,8968,8926,f283,52d5,8435,0d1f
0,OK,[],289 | 8TH ST / RED RIVER ST,[],GOOD,no,no,OK,Emergency Vehicle,OK,...,[],09:01,04:47,Cleaned,Replaced,NaN,NaN,NaN,NaN,NaN
1,OK,NaN,238 | CESAR CHAVEZ ST / WALTER SEAHOLM DR,OK,GOOD,no,no,OK,No Preemption,OK,...,OK,10:43,06:10,Cleaned,Replaced,Cleaned,"[{'updated_at': '1536232285', 'created_locatio...",NaN,NaN,NaN
2,OK,NaN,229 | CESAR CHAVEZ ST / SANDRA MURAIDA WAY,OK,GOOD,no,no,OK,No Preemption,OK,...,OK,09:23,04:40,Cleaned,Replaced,Cleaned,"[{'updated_at': '1536230172', 'created_locatio...",NaN,NaN,NaN
3,OK,NaN,440 | NORTHLAND DR / BALCONES DR,OK,GOOD,no,no,OK,No Preemption,OK,...,OK,11:08,10:24,Cleaned,OK,[],"[{'updated_at': '1536074767', 'created_locatio...",NaN,NaN,NaN
4,OK,NaN,675 | ANDERSON MILL RD / PARMER LN,OK,GOOD,yes,no,[],No Preemption,OK,...,OK,07:45,07:22,[],OK,Cleaned,"[{'updated_at': '1536063808', 'created_locatio...",1.Supervisor needs to check if cabinet is in n...,{'photo_id': '44fdb071-0b6c-482d-b738-7538ef6c...,NaN


In [106]:
type(records["6080"][0])

list